### Full Factorial Design

First steps in DOE

In [ ]:
%matplotlib widget
# Dependencies
import matplotlib.pyplot as plt
import numpy as np

# Settings
np.set_printoptions(precision=3)

In [ ]:
num = 3
x1 = np.linspace(-1,1,num=num)
x2 = np.linspace(-1,1,num=num)
x1_, x2_ = np.meshgrid(x1, x2)

def measure_response(x_1, x_2, noise=True):
    a =  0.5  # 0.3, -1.0
    b = -0.5  # 0.7,  0.2
    c =  2.0  # 1.0, -1.5
    y_1 = a*x_1 + b*x_2 + c*x_1*x_2
    if noise:
        y_1 += np.random.normal(0, 0.1, y_1.shape)
    return y_1

y1 = measure_response(x1_, x2_, False)

fig = plt.figure(figsize=(3,3))
ax = fig.add_subplot(1,1,1,projection='3d')
ax.set_proj_type('ortho')
ax.view_init(30, 150)
ax.set_xlim(-1,1)
ax.set_ylim(-1,1)
ax.set_zlim(np.min(y1),np.max(y1))
ax.set_xticks([-1,0,1])
ax.set_yticks([-1,0,1])
ax.set_zticks([np.min(y1),0,np.max(y1)])
ax.grid(False)
ax.set_xlabel('Factor 1')
ax.set_ylabel('Factor 2')
ax.set_zlabel('Response')
ax.plot_wireframe(x1_, x2_, y1*0, linewidth=1, color='k', alpha=0.5)
ax.scatter(x1_, x2_, y1*0, s=15, color='k', alpha=0.5)
ax.plot_wireframe(x1_, x2_, y1, linewidth=1, color='k')
ax.scatter(x1_, x2_, y1, s=15, color='r', alpha=1)

for xi, yi, zi in zip(x1_.flatten(), x2_.flatten(), y1.flatten()):
    ax.plot([xi, xi], [yi, yi], [0, zi], color='k', alpha=0.2, linewidth=1)

In [ ]:
""" Understanding OFAT vs Full Factor """
# OFAT design
x1m = np.r_[np.ones_like(x1)*x1[1], x1]
x2m = np.r_[x2, np.ones_like(x2)*x2[1]]
print("OFAT\n", np.c_[x1m, x2m])
# Full Factor design
x1m = np.repeat(x1, x1.shape[0])
x2m = np.tile(x2, x2.shape[0])
print("Full Factor\n", np.c_[x1m, x2m])

y1m = measure_response(x1m, x2m)

fig = plt.figure(figsize=(3,3))
ax = fig.add_subplot(1,1,1,projection='3d')
ax.set_proj_type('ortho')
ax.view_init(30, 150)
ax.set_xlim(-1,1)
ax.set_ylim(-1,1)
ax.set_zlim(np.min(y1),np.max(y1))
ax.set_xticks([-1,0,1])
ax.set_yticks([-1,0,1])
ax.set_zticks([np.min(y1),0,np.max(y1)])
ax.grid(False)
ax.set_xlabel('Factor 1')
ax.set_ylabel('Factor 2')
ax.set_zlabel('Response')
ax.scatter(x1m, x2m, y1m, color='c', alpha=1)

In [ ]:
""" Model regression from measured data """
from sklearn.linear_model import LinearRegression

# Design matrix for second-order model
X = np.column_stack((np.ones(x1m.shape),  # intercept
                     x1m,
                     x2m,
                     x1m*x2m))

# Fit regression
model = LinearRegression(fit_intercept=False)  # intercept already in X
model.fit(X, y1m)
betas = model.coef_

# Output coefficients
coeff_labels = ['β0', 'β1 (x)', 'β2 (y)', 'β3 (xy)']
for label, coef in zip(coeff_labels, betas):
    print(f'{label}: {coef:.4f}')

x1m_grid, x2m_grid = np.meshgrid(x1m, x2m)
y1m_grid = (betas[0] +
            betas[1]*x1m_grid +
            betas[2]*x2m_grid +
            betas[3]*x1m_grid*x2m_grid)

fig = plt.figure(figsize=(3,3))
ax = fig.add_subplot(1,1,1,projection='3d')
ax.set_proj_type('ortho')
ax.view_init(30, 150)
ax.set_xlim(-1,1)
ax.set_ylim(-1,1)
ax.set_zlim(np.min(y1),np.max(y1))
ax.set_xticks([-1,0,1])
ax.set_yticks([-1,0,1])
ax.set_zticks([np.min(y1),0,np.max(y1)])
ax.grid(False)
ax.set_xlabel('Factor 1')
ax.set_ylabel('Factor 2')
ax.set_zlabel('Response')
ax.plot_wireframe(x1m_grid, x2m_grid, y1m_grid*0, linewidth=1, color='k', alpha=0.5)
ax.scatter(x1m_grid, x2m_grid, y1m_grid*0, s=15, color='k', alpha=0.5)
ax.plot_wireframe(x1m_grid, x2m_grid, y1m_grid, linewidth=1, color='k')
ax.scatter(x1m_grid, x2m_grid, y1m_grid, s=15, color='c', alpha=1)
ax.scatter(x1_, x2_, y1, s=15, color='r', alpha=1)

#### Toy Example
Let's better understand how to DOE

In [ ]:
import pandas as pd

# Factor levels
A_levels = np.array(['M1', 'M2'])        # 2 levels
B_levels = np.array([4, 6, 8])     # 3 levels
C_levels = np.array([6000, 65000])        # 2 levels

# Number of levels per factor
n_A = len(A_levels)
n_B = len(B_levels)
n_C = len(C_levels)

# Total runs
n_runs = n_A * n_B * n_C

# Create the design matrix columns
# For A: repeat each level for (B*C) times, then tile for n_A times
A = np.repeat(A_levels, n_B * n_C)

# For B: repeat each level for n_C times, tile for n_A times
B = np.tile(np.repeat(B_levels, n_C), n_A)

# For C: tile levels for (n_A * n_B) times
C = np.tile(C_levels, n_A * n_B)

# Stack columns into a design matrix
design = np.column_stack((A, B, C))

# Convert to DataFrame for readability
df = pd.DataFrame(design, columns=['A', 'B', 'C'])
display(df)

In [ ]:
# Define true effects (arbitrary example)
# y = intercept + A_effect + B_effect + C_effect + interactions + noise
# def simulate_response(A, B, C):
#     return (5
#             + 2 * A                # effect of A
#             + 3 * B                # effect of B
#             - 1.5 * C              # effect of C
#             + 1.2 * A * B          # interaction A*B
#             + 0.5 * B * C          # interaction B*C
#             + np.random.normal(0, 0.5))  # noise

# Apply function row-wise
# df['y'] = [simulate_response(a, b, c) for a, b, c in design]
df['y'] = [0.36,0.47,0.39,0.53,0.46,0.74,0.37,0.49,0.41,0.55,0.48,0.76]
display(df)

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Convert numeric factors to categorical strings (IMPORTANT!)
df['A_cat'] = pd.Categorical(df['A'].astype(str))
# df['B_cat'] = pd.Categorical(df['B'].astype(str))
# df['C_cat'] = pd.Categorical(df['C'].astype(str))

# Check for NaNs and infs in response and factors
# print(df.isna().sum())
# print(np.isfinite(df['y']).all())

# Fit model with all interactions using categorical variables
model = ols('y ~ A_cat + B + C + A_cat:B + A_cat:C + B:C', data=df).fit()
print(model.params)

# Perform an ANOVA (analysis of variance) (type II sums of squares)
anova_table = sm.stats.anova_lm(model, typ=2)
display(anova_table)

In [ ]:
import seaborn as sns

factors = ['A_cat', 'B', 'C']
response = 'y'

# Set up subplot grid size (main effects + pairwise interactions)
n = len(factors)
fig, axes = plt.subplots(n, n, figsize=(7, 7), sharey=False)

for i, f1 in enumerate(factors):
    for j, f2 in enumerate(factors):
        ax = axes[i, j]
        if i == j:
            # Main effect plot: factor f1 on x-axis
            sns.pointplot(x=f1, y=response, data=df, errorbar='sd', ax=ax)
            ax.set_title(f'Main effect: {f1}')
            ax.set_xlabel(f1)
            ax.set_ylabel(response)
        else:
            # Interaction plot: f2 on x-axis, f1 as hue
            sns.pointplot(x=f2, y=response, hue=f1, data=df, errorbar='sd', ax=ax)
            ax.set_title(f'Interaction: {f1} x {f2}')
            ax.set_xlabel(f2)
            ax.set_ylabel(response)
            if j == 0:
                ax.legend(title=f1)
            else:
                ax.get_legend().remove()

plt.tight_layout()
plt.show()

#### Industrial Example: Machining

Response: Roughness

Table of factors

|Symbol|Factor|Level 1|Level 2|Level 3|
|---|---|---|---|---|
|MEU|Kind of grindstone|1|2| |
|ROT|Rotation of the part|4|6|8|
|VIT|Grindstone speed|6000|6500| |

Measured data

|MEU|ROT|VIT|Roughness|
|---|---|---|---|
|M1|4|6000|0.36|
|M1|4|6500|0.47|
|M1|6|6000|0.39|
|M1|6|6500|0.53|
|M1|8|6000|0.46|
|M1|8|6500|0.74|
|M2|4|6000|0.37|
|M2|4|6500|0.49|
|M2|6|6000|0.41|
|M2|6|6500|0.55|
|M2|8|6000|0.48|
|M2|8|6500|0.76|